In [1]:
import cv2
import numpy as np
import math


In [3]:
def _crop_to_signature(gray, thresh_val=220):
    # ... your code ...
    _, th = cv2.threshold(gray, thresh_val, 255, cv2.THRESH_BINARY_INV)
    coords = cv2.findNonZero(th)
    if coords is None:
        return gray
    x, y, w, h = cv2.boundingRect(coords)
    return gray[y:y+h, x:x+w]

def _deskew(gray):
    # ... your code ...
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    coords = np.column_stack(np.where(th > 0))
    if coords.size == 0:
        return gray
    rect = cv2.minAreaRect(coords.astype(np.float32))
    angle = rect[-1]
    if angle < -45:
        angle = 90 + angle
    if abs(angle) < 1:
        return gray
    (h, w) = gray.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray, M, (w, h),
                             flags=cv2.INTER_CUBIC,
                             borderMode=cv2.BORDER_REPLICATE)
    return rotated

def preprocess_signature(
    img_input,
    size=224,
    mean=0.5,
    std=0.5,
    deskew=True,
    crop=True,
    as_tensor=False
):
    # 1. Load image as grayscale
    if isinstance(img_input, str):
        gray = cv2.imread(img_input, cv2.IMREAD_GRAYSCALE)
        if gray is None:
            raise ValueError(f"Could not read image from path: {img_input}")
    else:
        img = img_input
        if len(img.shape) == 3 and img.shape[2] == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif len(img.shape) == 3 and img.shape[2] == 1:
            gray = img[:, :, 0]
        else:
            gray = img.copy()

    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    if crop:
        gray = _crop_to_signature(gray)

    if deskew:
        gray = _deskew(gray)

    h, w = gray.shape
    if h == 0 or w == 0:
        raise ValueError("Empty image encountered after cropping/deskewing.")

    scale = size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(gray, (new_w, new_h), interpolation=cv2.INTER_AREA)

    h2, w2 = resized.shape
    pad_top = (size - h2) // 2
    pad_bottom = size - h2 - pad_top
    pad_left = (size - w2) // 2
    pad_right = size - w2 - pad_left

    padded = cv2.copyMakeBorder(
        resized,
        pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=255
    )

    img_float = padded.astype(np.float32) / 255.0
    img_norm = (img_float - mean) / std

    if as_tensor:
        img_norm = np.expand_dims(img_norm, axis=0)  # (1,H,W)

    return img_norm


## Model

In [4]:

import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import math


In [8]:

# =========================
# 1. Embedding Backbone
# =========================
def build_embedding_model(input_shape=(224, 224, 1), embedding_dim=512, train_backbone=True):
    """
    Builds an image -> embedding model.
    - Grayscale input
    - Converts to RGB
    - Uses ResNet50 backbone + Dense + BN + L2 norm
    """
    # Grayscale input
    inputs = layers.Input(shape=input_shape, name="signature_input")

    # Convert 1-channel grayscale to 3 channels (RGB-like)
    x_rgb = layers.Lambda(lambda t: tf.image.grayscale_to_rgb(t),
                          name="gray_to_rgb")(inputs)
    # Now x_rgb shape: (H, W, 3)

    # Standard ResNet50 backbone (NO input_tensor!)
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",  # you can set to None while debugging
        input_shape=(input_shape[0], input_shape[1], 3)
    )
    base_model.trainable = train_backbone

    x = base_model(x_rgb, training=train_backbone)
    x = layers.GlobalAveragePooling2D(name="global_avg_pool")(x)

    # Embedding head: Dense + BatchNorm + L2 normalization
    x = layers.Dense(embedding_dim, use_bias=False, name="embedding_dense")(x)
    x = layers.BatchNormalization(name="embedding_bn")(x)

    # ❗ IMPORTANT: wrap l2_normalize in a Lambda layer
    x = layers.Lambda(
        lambda t: tf.nn.l2_normalize(t, axis=1),
        name="l2_normalized_embedding"
    )(x)

    model = models.Model(inputs=inputs, outputs=x, name="signature_embedding_model")
    return model

# =========================
# 2. ArcFace layer
# =========================

class ArcMarginProduct(layers.Layer):
    def __init__(self, num_classes, s=30.0, m=0.5, easy_margin=False, **kwargs):
        super(ArcMarginProduct, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.s = s
        self.m = m
        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def build(self, input_shape):
        embedding_dim = input_shape[0][-1]
        self.W = self.add_weight(
            name='W',
            shape=(embedding_dim, self.num_classes),
            initializer='glorot_uniform',
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        embeddings, labels = inputs
        W = tf.nn.l2_normalize(self.W, axis=0)

        cos_theta = tf.matmul(embeddings, W)
        cos_theta = tf.clip_by_value(cos_theta, -1.0 + 1e-7, 1.0 - 1e-7)

        sin_theta = tf.sqrt(1.0 - tf.square(cos_theta))
        cos_theta_m = cos_theta * self.cos_m - sin_theta * self.sin_m

        if self.easy_margin:
            cond = tf.cast(tf.greater(cos_theta, 0), tf.float32)
            cos_theta_m = cond * cos_theta_m + (1.0 - cond) * cos_theta
        else:
            cond_v = cos_theta - self.th
            cond = tf.cast(tf.nn.relu(cond_v), tf.bool)
            keep = tf.where(cond, cos_theta_m, cos_theta - self.mm)
            cos_theta_m = keep

        labels_one_hot = tf.one_hot(labels, depth=self.num_classes)
        logits = self.s * (labels_one_hot * cos_theta_m + (1.0 - labels_one_hot) * cos_theta)
        return logits

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "num_classes": self.num_classes,
            "s": self.s,
            "m": self.m,
            "easy_margin": self.easy_margin
        })
        return cfg


def build_arcface_training_model(num_classes, input_shape=(224, 224, 1), embedding_dim=512):
    embedding_model = build_embedding_model(
        input_shape=input_shape,
        embedding_dim=embedding_dim
    )

    image_input = layers.Input(shape=input_shape, name="image_input")
    label_input = layers.Input(shape=(), name="label_input", dtype=tf.int32)

    embeddings = embedding_model(image_input)
    logits = ArcMarginProduct(num_classes=num_classes, s=30.0, m=0.5)(
        [embeddings, label_input]
    )

    training_model = models.Model(
        inputs=[image_input, label_input],
        outputs=logits,
        name="arcface_signature_model"
    )

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    training_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=loss_fn,
        metrics=["accuracy"]
    )

    return training_model, embedding_model


In [9]:
num_classes = 100   # must match your actual number of unique signer IDs
input_shape = (224, 224, 1)

training_model, embedding_model = build_arcface_training_model(
    num_classes=num_classes,
    input_shape=input_shape,
    embedding_dim=512
)


In [10]:
import os
import glob

full_org_dir = "signatures/full_org"   # 🔴 change to your path

# get all images (png/jpg – adjust if needed)
image_paths = sorted(glob.glob(os.path.join(full_org_dir, "*.png")))

def parse_signer_id_from_filename(path):
    """
    Example:
      'original_1_1.png' -> signer 0
      'original_2_3.png' -> signer 1
    Pattern assumed: <prefix>_<signerId>_<sampleId>.png
    """
    fname = os.path.basename(path)       # 'original_1_1.png'
    parts = fname.split("_")             # ['original', '1', '1.png']
    signer_str = parts[1]                # '1'
    signer_int = int(signer_str) - 1     # make 0-based: 0,1,2,...
    return signer_int

labels = [parse_signer_id_from_filename(p) for p in image_paths]

num_classes = len(set(labels))
print("Total images:", len(image_paths))
print("Unique signers (classes):", num_classes)


Total images: 1320
Unique signers (classes): 55


In [11]:
path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))

def load_and_preprocess(path, label):
    def _preprocess(path_bytes):
        path_str = path_bytes.decode("utf-8")
        img = preprocess_signature(
            img_input=path_str,
            size=224,
            mean=0.5,
            std=0.5,
            deskew=True,
            crop=True,
            as_tensor=True,
        )
        if img.ndim == 3 and img.shape[0] == 1:
            img = np.transpose(img, (1, 2, 0))  # (H,W,1)
        return img.astype(np.float32)

    img = tf.numpy_function(_preprocess, [path], tf.float32)
    img.set_shape((224, 224, 1))

    label = tf.cast(label, tf.int32)

    return {"image_input": img, "label_input": label}, label

train_ds = (
    path_ds
    .shuffle(len(image_paths))
    .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:
training_model.fit(train_ds, epochs=5,batch_size=32)


Epoch 1/5
 6/42 ━━━━━━━━━━━━━━━━━━━━ 44:59 75s/step - accuracy: 0.0000e+00 - loss: 19.2532

In [ ]:
# Preprocess your image to shape (1, 224, 224, 1) float32
# x = preprocess_signature(...)

embedding = embedding_model.predict(x)  # shape: (1, 512), already L2-normalized
